In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lrex')\
        .master("local[*]")\
        .getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [6]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [7]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [8]:
lr = LinearRegression(featuresCol="features", labelCol="label"
                      , predictionCol='prediction')

In [9]:
lrModel = lr.fit(training)

In [10]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [11]:
lrModel.intercept

0.14228558260358093

In [12]:
training_summary = lrModel.summary

In [13]:
training_summary.r2

0.027839179518600154

In [14]:
training_summary.rootMeanSquaredError

10.16309157133015

In [15]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [18]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [21]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                350|
|   mean| 0.7930202951306546|
| stddev| 10.636901633140255|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                151|
|   mean|-0.9858000223578707|
| stddev|  9.454605171285463|
|    min|-26.736207182601724|
|    max| 20.000154367451547|
+-------+-------------------+



In [23]:
correct_model = lr.fit(train_data)

In [24]:
test_results = correct_model.evaluate(test_data)

In [25]:
test_results.rootMeanSquaredError

9.616372101466695

In [26]:
unlabeled_data = test_data.select('features')

In [27]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [28]:
predictions = correct_model.transform(unlabeled_data)

In [29]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|-0.7594568382920114|
|(10,[0,1,2,3,4,5,...| 0.4372145498490047|
|(10,[0,1,2,3,4,5,...| 1.2358890388688324|
|(10,[0,1,2,3,4,5,...| 1.6278109834225143|
|(10,[0,1,2,3,4,5,...|  2.109558280354943|
|(10,[0,1,2,3,4,5,...|0.23081818612274052|
|(10,[0,1,2,3,4,5,...|-0.7756300583740762|
|(10,[0,1,2,3,4,5,...| 1.5652024687310253|
|(10,[0,1,2,3,4,5,...|  2.449954671791089|
|(10,[0,1,2,3,4,5,...|  3.477069638961565|
|(10,[0,1,2,3,4,5,...| -1.610287753555339|
|(10,[0,1,2,3,4,5,...| 0.9345120748567597|
|(10,[0,1,2,3,4,5,...|-1.1363006429127172|
|(10,[0,1,2,3,4,5,...|-0.6168185270076315|
|(10,[0,1,2,3,4,5,...| 0.7446140688498402|
|(10,[0,1,2,3,4,5,...| 3.3130608019702605|
|(10,[0,1,2,3,4,5,...|0.36812238432251165|
|(10,[0,1,2,3,4,5,...|-3.4513695256111574|
|(10,[0,1,2,3,4,5,...|-0.5225465486831607|
|(10,[0,1,2,3,4,5,...| 0.7620763099626017|
+----------

In [30]:
spark.stop()